In [1]:
import argparse
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM
from advertorch.attacks import LinfSPSAAttack
from trainers import Trainer, FGSMTrainer
from robustbench.model_zoo.models import Carmon2019UnlabeledNet
from utils import adversarial_accuracy, fgsm_
import eagerpy as ep
from Nets import CIFAR_Wide_Res_Net, CIFAR_Res_Net, CIFAR_Net
from gradient_masking_tests import run_masking_benchmarks
%load_ext autoreload
%autoreload 2
%aimport Nets, trainers

## Setup to import trained CIFAR-10 Models

In [2]:
# setup
device = torch.device("cuda")
batch_size = 32
# remove the normalize
transform = transform = transforms.Compose(
            [transforms.ToTensor()]
)
        
normalized_min = (0 - 0.5) / 0.5
normalized_max = (1 - 0.5) / 0.5
train_dataset = datasets.CIFAR10(root='./data', train=True,
                                download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                  shuffle=True, num_workers=2)
test_dataset = datasets.CIFAR10(root='./data', train=False,
                               download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                 shuffle=False, num_workers=2)
classes = classes = ('plane', 'car', 'bird', 'cat',
   'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load Regular CIFAR-10 Model

In [3]:
model = CIFAR_Net(device).eval()
model.load_state_dict(torch.load("models/cifar_simple_net_50e.model"))

<All keys matched successfully>

## Load CIFAR-10 Model trained with large FGSM steps

In [4]:
fgsm_model = CIFAR_Net(device).eval()
fgsm_model.load_state_dict(torch.load("models/cifar_simple_net_fgsm06_50e.model"))

<All keys matched successfully>

## RESNET Models

In [39]:
run_masking_benchmarks(model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

  0%|                                                                                          | 0/313 [00:00<?, ?it/s]

Model accuracy: 84.86%


  0%|                                                                                          | 0/313 [00:00<?, ?it/s]

FGSM attack model accuracy -- eps = 0.006: 36.870000000000005%, eps = 0.03: 4.58%, eps = 0.06: 1.87%


  0%|                                                                                          | 0/313 [00:00<?, ?it/s]

PGD accuracy: 0.0%


  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

Unbounded PGD model accuracy: 0.0%


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.71s/it]

SPSA accuracy: 15.0%


In [40]:
run_masking_benchmarks(fgsm_model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.71s/it]

Model accuracy: 72.53%
FGSM attack model accuracy -- eps = 0.006: 10.299999999999999%, eps = 0.03: 73.95%, eps = 0.06: 85.24000000000001%
Gradient Masking Warning: Increasing epsilon did not improve the attack!!
PGD accuracy: 0.0%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 5.0%
Gradient Masking Warning: Black Box attack was stronger than FGSM attack!!


## Simple models

In [5]:
run_masking_benchmarks(model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  3.03it/s]

Model accuracy: 68.76%
FGSM attack model accuracy -- eps = 0.006: 35.5%, eps = 0.03: 19.23%, eps = 0.06: 16.07%
PGD accuracy: 2.1%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 24.0%


In [6]:
run_masking_benchmarks(fgsm_model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  2.95it/s]

Model accuracy: 48.16%
FGSM attack model accuracy -- eps = 0.006: 46.52%, eps = 0.03: 36.230000000000004%, eps = 0.06: 27.529999999999998%
PGD accuracy: 21.88%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 31.0%
